In [15]:
# Importing packages
import os
import yaml
import logging
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata
import qiime2.plugins.dada2.actions as dada2_actions
import qiime2.plugins.metadata.actions as metadata_actions
from qiime2.plugins.feature_table.visualizers import tabulate_seqs
from qiime2.plugins.feature_table.visualizers import summarize
from qiime2.plugins.feature_table.visualizers import core_features

from qiime2.plugins.feature_table.methods import merge
from qiime2.plugins.feature_table.methods import merge_seqs
from qiime2.plugins.feature_table.methods import merge_taxa

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

from qiime2 import Artifact, Visualization
from qiime2.plugins import demux
from qiime2.plugins.cutadapt.methods import trim_paired

import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
# Parameters
experiment_name = "thayane-PM"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-thayane"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-thayane/data/metadata.tsv"

In [12]:
# Define paths
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
data_folder = os.path.join(base_dir, 'data')
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

demux_single_path = os.path.join(data_folder, 'manifest-single.csv')
demux_paired_path = os.path.join(data_folder, 'manifest-paired.csv')

In [17]:
# demux_single = artifact = Artifact.import_data(
#         'SampleData[SequencesWithQuality]', 
#         demux_single_path, 
#         view_type='SingleEndFastqManifestPhred33')
# demux_single.save(os.path.join(qiime_folder, 'demux-single.qza'))


demux_paired = artifact = Artifact.import_data(
        'SampleData[PairedEndSequencesWithQuality]', 
        demux_paired_path, 
        view_type='PairedEndFastqManifestPhred33')
demux_paired.save(os.path.join(qiime_folder, 'demux-paired.qza'))

'/home/lauro/nupeb/rede-micro/redemicro-thayane/experiments/thayane-PM/qiime-artifacts/demux-paired.qza'

In [ ]:
Visualization.save(demux.visualizers.summarize(demux_single).visualization, filepath = os.path.join(
    qiime_folder, 
    'demux-single.qzv'))

Visualization.save(demux.visualizers.summarize(demux_paired).visualization, filepath = os.path.join(
    qiime_folder, 
    'demux-paired.qzv'))

In [ ]:
forward_primer = 'CCTACGGGRSGCAGCAG'
reverse_primer = 'GGACTACHVGGGTWTCTAAT'
forward_reverse_complement = [str(Seq(forward_primer[0]).reverse_complement())]
reverse_reverse_complement = [str(Seq(reverse_primer[0]).reverse_complement())]

trim_paired_seqs = trim_paired(
    demultiplexed_sequences=artifact,
    front_f=forward_primer,
    front_r=reverse_primer,
#     adapter_f=reverse_reverse_complement,
#     adapter_r=forward_reverse_complement,
    cores=6,
    overlap=12,
    indels=False,
    match_read_wildcards=True,
    match_adapter_wildcards=True,
    error_rate=0.15,
    discard_untrimmed=True,
).trimmed_sequences